# Decoding with Belief Propagation

We want to iterate the following the following equations:

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu } \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$  

which are Eqs.(96) from the paper [Low-density parity-check codes—A statistical physics perspective](https://www.sciencedirect.com/science/article/pii/S1076567002800180), by R. Vicente, D. Saad and Y. Kabashima.

The function $ \beta(x) $ is the Nishimori temperature,

$$ \beta(x) = \frac{1}{2} \log \left(  \frac{1- \rho}{\rho}  \right) \; \; .  $$

The quantity $\rho$ is the flip probablility of the noisy channel (BSC),

$$ P ( J | J^{(0)} ) = (1 - \rho) \delta_{J, J^{(0)} } + \rho \delta_{J, -J^{(0)} }   \; \; , $$

whereas the prior distribution for each meassage bit is assumed to be

$$ P ( S_j ) = (1 - \rho_\xi) \delta_{+1, S_j }  +  \rho_\xi \delta_{-1, S_j }  \; \; .  $$

The object $ {\cal L} (\mu)$ represents the set of $K$ non-zero elements on the row $\mu$ of the code generator matrix ${\cal G}$ (the one which adds redundancy), 

$$  {\cal L} (\mu) = \langle i_1, i_2, ..., i_K \rangle  \; \; .  $$

The are $C$ non-zero elements per column on the matrix ${\cal G}$:

$$ \sum_{\mu : j \in {\cal L}(\mu)} i_j = C \; \; ; \; \; \forall j = 1, ..., K \; \; . $$

The object $ {\cal M} (j)$ represents the set of all index sets that contain $j$.

After convergence of the iterative procedure, we can calculate the pseudo-posterior:

$$ m_{j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) } \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$
from which the Bayes optimal estimate is obtained
$$ \hat{\xi}_j  = sign( m_j ) $$ 

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

Defining the variables of the problem:

In [59]:
# Message lengh
N = 100

# Codeword lengh
M = 200

# Non-zero elements per row of the generation matrix
K = 4

# Number of messages
n = 10

# Noisy channel
p = 0.3
beta = 0.5*np.log( (1 - p) / p)

# Message prior
p_prior = 0.1
beta_prior = 0.5*np.log( (1 - p_prior) / p_prior)

## Generating messages

Each message is a $N$ dimensional vector. Generate a set of $n$ messages.

In [60]:
random = torch.rand([n, N])
    
message = torch.zeros([n, N])

for j in range(random.shape[0]):
    
    for k in range(random.shape[1]):
               
            #### -1 with probability p_prior
            if random[j,k] <= p_prior:
                message[j,k] = -1.
            #### +1 with probability 1 - p_prior
            else:
                message[j,k] = +1.

## Encoding

Each message is encoded to a high dimensional vector ${\bf J}^{(0)} \in \{ \pm 1  \}^M$ defined as 

$$   J^{(0)}_{\langle i_1, i_2, ...., i_K \rangle} = \xi_1 \xi_ 2 ... \xi_K  \; \; ,$$

where $M$ sets of $K \in [ 1, ..., N]$ indexes are randomly chosen.

In [61]:
encoding = torch.randint(0, N, [M, K])

In [62]:
encoding.shape

torch.Size([200, 4])

From `encoding`, we construct the encoded message ${\bf J}^{(0)}$.

In [63]:
# Initializing
J0 = torch.take(message[0], encoding).prod(dim=1)
J0 = J0.unsqueeze(0)

for j in range(1, message.shape[0]):
    
    J0_ = torch.take(message[j], encoding).prod(dim=1)
    J0_ = J0_.unsqueeze(0)
    
    J0 = torch.cat((J0, J0_), dim= 0)

## Corrupted version of the encoded set of messages

In [64]:
J = J0.clone()

In [65]:
random = torch.rand(J.shape)
                      
for j in range(J.shape[0]):
    for k in range(J.shape[1]):
          
        if random[j, k] <= p:
            J[j, k] = -J[j, k]

In [66]:
J

tensor([[ 1., -1.,  1.,  ...,  1.,  1., -1.],
        [-1., -1.,  1.,  ...,  1.,  1., -1.],
        [-1.,  1.,  1.,  ..., -1., -1., -1.],
        ...,
        [ 1.,  1.,  1.,  ...,  1.,  1., -1.],
        [ 1., -1.,  1.,  ...,  1.,  1., -1.],
        [ 1., -1., -1.,  ..., -1.,  1., -1.]])

In [67]:
J.shape

torch.Size([10, 200])

In [74]:
## Flip tensor: -1 with probability p
flip = 2*(random > p).float() - 1

## J corrupted version: element wise multiplication
J_ = torch.mul(J0, flip)

In [75]:
p

0.3

In [76]:
flip

tensor([[ 1., -1.,  1.,  ...,  1.,  1., -1.],
        [ 1., -1.,  1.,  ...,  1.,  1., -1.],
        [ 1.,  1.,  1.,  ...,  1., -1., -1.],
        ...,
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1., -1.],
        [-1.,  1., -1.,  ...,  1.,  1.,  1.]])

In [72]:
random

tensor([[0.9124, 0.1064, 0.6410,  ..., 0.6197, 0.5000, 0.0101],
        [0.4607, 0.0198, 0.5395,  ..., 0.9540, 0.7226, 0.1243],
        [0.8156, 0.9868, 0.3049,  ..., 0.6664, 0.1401, 0.2027],
        ...,
        [0.6110, 0.4401, 0.6442,  ..., 0.5178, 0.8706, 0.9719],
        [0.8922, 0.8729, 0.5421,  ..., 0.8211, 0.9470, 0.0958],
        [0.1390, 0.5615, 0.2017,  ..., 0.6895, 0.7020, 0.8423]])

In [73]:
random <= p

tensor([[False,  True, False,  ..., False, False,  True],
        [False,  True, False,  ..., False, False,  True],
        [False, False, False,  ..., False,  True,  True],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False,  True],
        [ True, False,  True,  ..., False, False, False]])

In [77]:
J == J_

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

## Iterating BP equations for one message

Let us focus in one received message to iterate the belief propagation equations.

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu} \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right) + \beta(\rho_\xi)    \right)   \; \; ,  $$  

with $j = 1, ..., N$ and $\mu = 1, ..., M$.

We cal this message `J_`. We will worry later about a loop over all the received messages.

In [14]:
J_ = J[0]
print(J_)
print(J_.shape)

tensor([ 1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1.,
         1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1., -1., -1.,  1.,  1.,
        -1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,  1.,
         1., -1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,
         1., -1.,  1., -1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1.,
        -1., -1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.,  1.,
         1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,  1., -1., -1., -1.,
         1., -1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1.,
        -1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,
        -1., -1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1.,  1., -1.,
        -1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1., -1.,
        -1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  

Random initialization of the beliefs $m_{\mu l}$.

In [15]:
m = torch.rand(M, N)

In [16]:
m

tensor([[0.6993, 0.7876, 0.6544,  ..., 0.1501, 0.7978, 0.7293],
        [0.8825, 0.2156, 0.6157,  ..., 0.6019, 0.1765, 0.9871],
        [0.8097, 0.8887, 0.5504,  ..., 0.5052, 0.8078, 0.1740],
        ...,
        [0.3305, 0.4320, 0.4681,  ..., 0.8783, 0.2415, 0.5931],
        [0.1749, 0.8697, 0.8215,  ..., 0.5397, 0.6670, 0.7277],
        [0.5649, 0.5952, 0.6937,  ..., 0.9226, 0.3572, 0.1892]])

Initialize an empty tensor to represent $\hat{m}_{\mu l}$.

In [17]:
m_hat = torch.empty(M, N)

In [18]:
m_hat

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

We want to calculate $\hat{m}_{\mu j}$.

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

This first implementation has two `for` loops. This is potentially harmful if one cares about efficienty. We obviously do, but since we are just beginning, lets go on like this.

In [19]:
for mu in range(M):
    for j in range(N):
          
        # Keep only L(mu) which a are different of j
        index_no_j = torch.nonzero(encoding[mu] != j).squeeze()
        L_no_j = encoding[mu][index_no_j]
        
        # Message update        
        m_hat[mu, j] = torch.tanh( beta* J_[mu])*torch.take(m[mu], L_no_j).prod(dim=0)

In [20]:
m_hat

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [-0., -0., -0.,  ..., -0., -0., -0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

The next step is to implement:

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu} \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right) + \beta(\rho_\xi)     \right)  \; \; ,  $$ 

In [21]:
for j in range(N):
    
    for mu in range(M):
        
        M_set = torch.where(encoding == j)[0]
        
        M_set_no_mu = M_set[torch.nonzero(M_set != mu).squeeze()]
        
        m[mu, j] = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set_no_mu).sum() + beta_prior)

In [22]:
m

tensor([[0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        ...,
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000]])

The next step is to implement the pseudo-posterior, which is calculated after convergence of the messages above:

$$ m_{j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) } \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$

In [23]:
m_bayes = []

for j in range(N):
    
    M_set = torch.where(encoding == j)[0]
       
    m_j = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set).sum() + beta_prior)
    
    m_bayes.append(m_j.item())
            
m_bayes

[0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,


In [24]:
len(m_bayes)

100

In [25]:
np.sign(m_bayes)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Gathering all message iterations on functions:

In [26]:
def m_to_mhat(N, M, J_, m, beta, encoding):
    
    m_hat = torch.empty(M, N)
    
    for mu in range(M):
        for j in range(N):
                     
            # Keep only L(mu) which a are different of j
            index_no_j = torch.nonzero(encoding[mu] != j).squeeze()
            L_no_j = encoding[mu][index_no_j]
        
            # Message update        
            m_hat[mu, j] = torch.tanh( beta* J_[mu])*torch.take(m[mu], L_no_j).prod(dim=0)
            
    return m_hat


#####################################################################


def mhat_to_m(N, M, m_hat, beta_prior, encoding):
    
    m = torch.empty(M, N)
    
    for j in range(N):
        
        for mu in range(M):
            
            M_set = torch.where(encoding == j)[0]
        
            M_set_no_mu = M_set[torch.nonzero(M_set != mu).squeeze()]
            
            m[mu, j] = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set_no_mu).sum() + beta_prior)
            
    return m


#####################################################################


def m_bayes(N, m_hat_final, beta_prior, encoding):
    
    m_bayes = []

    for j in range(N):
           
        M_set = torch.where(encoding == j)[0]
  
        m_j = torch.tanh(torch.take(np.arctanh(m_hat_final[:, j]), M_set).sum() + beta_prior)
    
        m_bayes.append(m_j.item())
        
    return m_bayes

Now we construct a function for the iterative decoding.

In [27]:
def BP_LDPC(N, M, J_, beta, beta_prior, encoding, message, num_it= 100, verbose= 1):
     
    m_hat = torch.rand(M, N)
    m = torch.empty(M, N)
    
    for j in range(num_it):
        
        print('--it = %d' % j)
              
        m = mhat_to_m(N, M, m_hat, beta_prior, encoding)
        m_hat = m_to_mhat(N, M, J_, m, beta, encoding)
        
        ## Monitoring performace
        if (j % verbose) == 0:
            m_ = m_bayes(N, m_hat, beta_prior, encoding)
            print('overlap= ', torch.dot(message, torch.Tensor(np.sign(m_))).item() / N)
        
    m_final = m_bayes(N, m_hat, beta_prior, encoding)
    
    return np.sign(m_final)

In [28]:
opt_dec_Bayes = BP_LDPC(N, M, J_, beta, beta_prior, encoding, message[0], num_it= 20, verbose= 1)

--it = 0
overlap=  0.82
--it = 1
overlap=  0.82
--it = 2
overlap=  0.82
--it = 3
overlap=  0.82
--it = 4
overlap=  0.82
--it = 5
overlap=  0.82
--it = 6
overlap=  0.82
--it = 7
overlap=  0.82
--it = 8
overlap=  0.82
--it = 9
overlap=  0.82
--it = 10
overlap=  0.82
--it = 11
overlap=  0.82
--it = 12
overlap=  0.82
--it = 13
overlap=  0.82
--it = 14
overlap=  0.82
--it = 15
overlap=  0.82
--it = 16
overlap=  0.82
--it = 17
overlap=  0.82
--it = 18
overlap=  0.82
--it = 19
overlap=  0.82


Ok. It is still slow (we will worrie about this latter), but it appears to work.

Observe that we arbitrarily consider a certain `num_it`. 

## Quick test on previous codes and messages

In [157]:
# Message lengh
N = 250

# Codeword lengh
M = 500

# Non-zero elements per row of the generation matrix
K = 4

# Number of messages
n = 100

# Noisy channel
p = 0.3
beta = 0.5*np.log( (1 - p) / p)

# Message prior
p_prior = 0.01
beta_prior = 0.5*np.log( (1 - p_prior) / p_prior)

In [136]:
encoding = torch.load('codes/code_N_250_M_500_K_4_p_03_p_prior_001_j_0.pt')
message = torch.load('codes/message_N_250_M_500_K_4_p_03_p_prior_001.pt')

In [137]:
encoding

tensor([[ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        ...,
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]])

In [145]:
print(message)
message.shape

tensor([[ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        ...,
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]])


torch.Size([100, 250])

Well, we made a mistake saving the codes. We have actually saved the messages. That is a problem. For now, let us generate a new code.

In [139]:
encoding  = torch.randint(0, N, [M, K])
encoding

tensor([[ 94,  21, 204, 242],
        [162,  49, 100, 151],
        [230, 198, 123,  57],
        ...,
        [ 87, 176,   1, 211],
        [ 99,  72,  51,  91],
        [248, 145, 105, 154]])

Encoding.

In [140]:
# Initializing
J0 = torch.take(message[0], encoding).prod(dim=1)
J0 = J0.unsqueeze(0)

for j in range(1, message.shape[0]):
    
    J0_ = torch.take(message[j], encoding).prod(dim=1)
    J0_ = J0_.unsqueeze(0)
    
    J0 = torch.cat((J0, J0_), dim= 0)

In [144]:
print(J0)
J0.shape

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])


torch.Size([100, 500])

Corrupted version.

In [142]:
J = J0.clone()

random = torch.rand(J.shape)
                      
for j in range(J.shape[0]):
    for k in range(J.shape[1]):
          
        if random[j, k] <= p:
            J[j, k] = -J[j, k]

In [143]:
print(J)
J.shape

tensor([[-1., -1.,  1.,  ...,  1., -1.,  1.],
        [-1.,  1., -1.,  ...,  1.,  1.,  1.],
        [ 1., -1., -1.,  ..., -1.,  1.,  1.],
        ...,
        [-1.,  1.,  1.,  ..., -1.,  1.,  1.],
        [ 1.,  1., -1.,  ...,  1.,  1., -1.],
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.]])


torch.Size([100, 500])

In [149]:
J[0].shape

torch.Size([500])

In [151]:
message[0].shape

torch.Size([250])

In [155]:
print(N)
print(M)
print(encoding.shape)
print(J.shape)
print(message.shape)

250
500
torch.Size([500, 4])
torch.Size([100, 500])
torch.Size([100, 250])


In [170]:
overlap_list = []

for k in range(int(J.shape[0] / 10 )):
    
    print('----message %d' % k)
       
    opt_dec_Bayes = BP_LDPC(N, M, J[k], beta, beta_prior, encoding, message[k], num_it= 10, verbose= 1)
    
    overlap_list.append(opt_dec_Bayes)

----message 0
--it = 0
overlap=  0.992
--it = 1
overlap=  0.992
--it = 2
overlap=  0.992
--it = 3
overlap=  0.992
--it = 4
overlap=  0.992
--it = 5
overlap=  0.992
--it = 6
overlap=  0.992
--it = 7
overlap=  0.992
--it = 8
overlap=  0.992
--it = 9
overlap=  0.992
----message 1
--it = 0
overlap=  0.984
--it = 1
overlap=  0.992
--it = 2
overlap=  0.984
--it = 3
overlap=  0.984
--it = 4
overlap=  0.984
--it = 5
overlap=  0.984
--it = 6
overlap=  0.984
--it = 7
overlap=  0.984
--it = 8
overlap=  0.984
--it = 9
overlap=  0.984
----message 2
--it = 0
overlap=  0.976
--it = 1
overlap=  0.976
--it = 2
overlap=  0.976
--it = 3
overlap=  0.976
--it = 4
overlap=  0.976
--it = 5
overlap=  0.976
--it = 6
overlap=  0.976
--it = 7
overlap=  0.976
--it = 8
overlap=  0.976
--it = 9
overlap=  0.976
----message 3
--it = 0
overlap=  0.968
--it = 1
overlap=  0.968
--it = 2
overlap=  0.968
--it = 3
overlap=  0.968
--it = 4
overlap=  0.968
--it = 5
overlap=  0.968
--it = 6
overlap=  0.968
--it = 7
overlap=  